<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module04/04_EDGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine-python/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: ED espaciales en Google Earth Engine III**
## **ee.Geometry, ee.Feature y ee.FeatureCollection**

En esta lectura, aprenderemos sobre:

- ee.ImageCollection como ED nativa de GEE.
- Filtar y mapear FeatureCollections.
- Puntos aleatorios


### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_AQUI_TU_TOKEN"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

<center>
<h3>GeoJSON: FeatureCollection  --> Feature --> Geometry --> Coordinates</h3>
</center>

### **3.  Geometrias en GEE (ee.Geometry)**


Earth Engine maneja datos vectoriales del tipo `Geometry`. La [especificacion de GeoJSON](https://geojson.org/geojson-spec.html) describe en detalle el tipo de geometrias admitidas por Earth Engine. Estas incluyen: `Point` (una lista de coordenadas en alguna proyeccion),`LineString` (una lista de puntos), `LinearRing` (un` LineString` cerrado) y `Polygon` (una lista de` LinearRings` donde el primero es un caparazon y los anillos posteriores son agujeros). Earth Engine tambien admite `MultiPoint`,` MultiLineString` y `MultiPolygon`. GeoJSON GeometryCollection tambien es compatible, aunque tiene el nombre `MultiGeometry` dentro de Earth Engine.

#### **Crear un objeto ee.Geometry**

Para crear una **geometria**, proporcione al constructor las listas de coordenadas adecuadas. Por ejemplo:

In [0]:
point = ee.Geometry.Point([1.5, 1.5])
lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])
linearRing = ee.Geometry.LinearRing([[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]])
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

En los ejemplos anteriores, tenga en cuenta que la distinción entre un "LineString" y un "LinearRing" es que el "LinearRing" está "cerrado" al tener la misma coordenada al principio y al final de la lista.

Para dividir una `geometria` compuesta en sus geometrias simples, use `geometry.geometries()`. Por ejemplo:

In [0]:
# Crear una geometria compuesta
multiPoint = ee.Geometry.MultiPoint([[-121.68, 39.91], [-97.38, 40.34]])

# Obtenga geometrias simples
geometries = multiPoint.geometries()

# Obtenga cada geometría individual de la lista e imprímala.
pt1 = geometries.get(0)
pt2 = geometries.get(1)
print('Point 1', pt1)
print('Point 2', pt2)

Point 1 ee.ComputedObject({
  "type": "Invocation",
  "arguments": {
    "list": {
      "type": "Invocation",
      "arguments": {
        "geometry": {
          "type": "MultiPoint",
          "coordinates": [
            [
              -121.68,
              39.91
            ],
            [
              -97.38,
              40.34
            ]
          ]
        }
      },
      "functionName": "Geometry.geometries"
    },
    "index": 0
  },
  "functionName": "List.get"
})
Point 2 ee.ComputedObject({
  "type": "Invocation",
  "arguments": {
    "list": {
      "type": "Invocation",
      "arguments": {
        "geometry": {
          "type": "MultiPoint",
          "coordinates": [
            [
              -121.68,
              39.91
            ],
            [
              -97.38,
              40.34
            ]
          ]
        }
      },
      "functionName": "Geometry.geometries"
    },
    "index": 1
  },
  "functionName": "List.get"
})


### **4. Operaciones geometricas**

Earth Engine admite una amplia variedad de operaciones en objetos `Geometry`. Estos incluyen operaciones en geometrias individuales, como calcular un búfer, centroide, cuadro delimitador, perimetro, casco convexo, etc. Por ejemplo:

In [0]:
# Crear un poligono
polygon = ee.Geometry.Polygon(coords = [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

# Obtener buffer
buffer = polygon.buffer(1000000).getInfo()

# Obtener el centroide del poligono
centroid = polygon.centroid().getInfo()

#Generar mapa!
mapdisplay(center=centroid['coordinates'],dicc={'buffer':buffer, 'centroid':centroid}, zoom_start=3)

### **5. Feature en GEE (ee.Feature)**
Un `Feature` en Earth Engine se define identicamente que un `Feature` de GeoJSON. Específicamente, un `Feature` es un objeto que almacena un objeto `geometria` (o nulo) y `propiedades` almacenadas en un diccionario.


#### **Creacion de objetos Features**

Para crear un `Feature`, proporcione una `geometria` y (opcionalmente) un diccionario de propiedades. Por ejemplo:

In [0]:
# Crear un ee.Gometry.Polygon
polygon = ee.Geometry.Polygon([[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]])

# Crear un Feature a partir de la geometria
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart','roi':'Sudamerica'})

Un `Feature` no necesita tener una geometria y puede simplemente envolver un diccionario de propiedades. Por ejemplo:

In [0]:
# Crear un diccionario de propiedades.
dict_01 = {'foo': ee.Number(8).add(88), 'bar': 'nihao'}
nowhereFeature = ee.Feature(None, dict_01)

Agrega nuevas propriedades usando el metodo `*.set`.

In [0]:
feature = ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]), {'genus':'Sequoia','species':'sempervirens'})
feature.get('species').getInfo()

'sempervirens'

In [0]:
from pprint import pprint

# Agrega propiedades usando el metodo set
feature = ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))\
            .set('genus', 'Sequoia').set('species', 'sempervirens')
feature.getInfo()
# Obtenga una propiedad de la caracteristica
species = feature.get('species')
print(species.getInfo())

# Establecer una nueva propiedad
feature = feature.set('presence', 1)

# Sobrescribir las propiedades antiguas con un nuevo diccionario
newDict = {'genus': 'Brachyramphus', 'species': 'marmoratus'}
feature = feature.set(newDict)

# Comprueba el resultado.
pprint(feature.getInfo())

sempervirens
{'geometry': {'coordinates': [-122.22599, 37.17605], 'type': 'Point'},
 'properties': {'genus': 'Brachyramphus',
                'presence': 1,
                'species': 'marmoratus'},
 'type': 'Feature'}


### **5. FeatureCollection en GEE (ee.FeatureCollection)**

Grupos de `Features` relacionados se pueden combinar en un `FeatureCollection`, para permitir operaciones adicionales en todo el conjunto de datos, como el filtrado, la clasificacion y el renderizado. Ademas de los `Features` (geometrias + propiedades), los `FeatureCollection` tambien pueden contener otras colecciones.

#### **Creacion de objetos FeatureCollection**
 
Una forma de crear una `FeatureCollection` es proporcionar al constructor una lista de `Features`. Los `Features` no necesitan tener el mismo tipo de geometria o las mismas propiedades. Por ejemplo:

In [0]:
# Haga una lista de features.
features = [
  ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
]

# Cree un ee.FeatureCollection a partir de la lista e imprimala.
fromList = ee.FeatureCollection(features)
fromList.getInfo()

{'columns': {'name': 'String', 'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[30.01, 59.8],
      [30.59, 59.8],
      [30.59, 60.15],
      [30.01, 60.15],
      [30.01, 59.8]]],
    'type': 'Polygon'},
   'id': '0',
   'properties': {'name': 'Voronoi'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.96, 40.781], 'type': 'Point'},
   'id': '1',
   'properties': {'name': 'Thiessen'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [6.4806, 50.8012], 'type': 'Point'},
   'id': '2',
   'properties': {'name': 'Dirichlet'},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

In [0]:
geom_01 = ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15)
geom_01_fc = ee.FeatureCollection([ee.Feature(geom_01,{})])
geom_01_fc.getInfo()

{'coordinates': [[[30.01, 59.8],
   [30.59, 59.8],
   [30.59, 60.15],
   [30.01, 60.15],
   [30.01, 59.8]]],
 'type': 'Polygon'}

### **6. Map sobre un FeatureCollection**

Para aplicar la misma operación a cada `Feature` en una` FeatureCollection`, use `featureCollection.map ()`. Por ejemplo, para agregar otro atributo de área a cada entidad en una `FeatureCollection` de cuencas,use:

In [0]:
from pprint import pprint

# Cargar los datos de cuencas
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06').limit(10)

# Esta funcion calcula el area de geometria de la entidad y la agrega como una propiedad.
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})

# Map! para obtener la nueva propiedad 'areaHa'
areaAdded = sheds.map(addArea)
centroid = sheds.geometry().centroid().coordinates().getInfo()

print('Primer feature: \n')
pprint(sheds.first().toDictionary().getInfo())

print('Primer feature: \n')
pprint(areaAdded.first().toDictionary().getInfo())

mapdisplay(center=centroid,
           dicc={'cuencas':sheds.draw(color = "FF0000").getMapId()},
           zoom_start=5)

Primer feature: 

{'areaacres': '17935746.85',
 'areasqkm': '72583.46',
 'gnis_id': '',
 'huc6': '190301',
 'loaddate': '20120611075458',
 'metasource': '',
 'name': 'Aleutian Islands',
 'shape_area': '9.9308571861444',
 'shape_leng': '81.5083456556245',
 'sourcedata': '',
 'sourcefeat': '',
 'sourceorig': '',
 'states': 'AK',
 'tnmid': '{7D8E9EC1-6A0D-4FED-8063-94688560AC75}'}
Primer feature: 

{'areaHa': 7227400.935025958,
 'areaacres': '17935746.85',
 'areasqkm': '72583.46',
 'gnis_id': '',
 'huc6': '190301',
 'loaddate': '20120611075458',
 'metasource': '',
 'name': 'Aleutian Islands',
 'shape_area': '9.9308571861444',
 'shape_leng': '81.5083456556245',
 'sourcedata': '',
 'sourcefeat': '',
 'sourceorig': '',
 'states': 'AK',
 'tnmid': '{7D8E9EC1-6A0D-4FED-8063-94688560AC75}'}


### **7. Muestras aleatorias**

Para obtener una coleccion de puntos aleatorios en una region especifica puede usar:

In [0]:
# Defina una region arbitraria en la que calcular puntos aleatorios.
region = ee.Geometry.Rectangle(-119.224, 34.669, -99.536, 50.064)
center_coord = region.centroid().coordinates().getInfo()

# Crea 1000 puntos aleatorios en la region.
randomPoints = ee.FeatureCollection.randomPoints(region=region,points=5000)

# Mostrar los puntos.
mapdisplay(center_coord,{'randomPoints':randomPoints.getInfo()},zoom_start=5)